Projet Data Mining

In [11]:
import requests

url = 'https://query.wikidata.org/sparql'
query = '''
SELECT ?itemLabel ?pic ?movementLabel ?creatorLabel ?inception
WHERE
{
?item wdt:P31 wd:Q3305213 .
?item wdt:P135 ?movement .
?item wdt:P170 ?creator .
?item wdt:P18 ?pic .
?item wdt:P571 ?inception
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
}
LIMIT 100
'''

r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()


In [14]:
import pandas as pd
import shutil, os

shutil.rmtree('./images')
os.mkdir('./images')
parsed_data = []

for entry in data ['results']['bindings']:
    url = entry['pic']['value']
    parsed_data.append([url,
     entry['movementLabel']['value'],
     entry['creatorLabel']['value'],
     entry['inception']['value']])
    #filename = url.split('/')[-1]
    #r = requests.get(url, allow_redirects=True)
    #open('./images/' + filename, 'wb').write(r.content)

df = pd.DataFrame(parsed_data,
     columns=['picture', 'movement', 'creator', 'inception'])
df = df.astype(dtype= {"picture":"<U200", "movement":"<U200", "creator":"<U200", "inception":"<U200"})


In [15]:
def recompute(selection, pMvt, V0, E0, V1, E1, iterations):
    # recalcule param en fct du choix

    # Pour les dates

    E0 += int(round(E0 - selection))/iterations
    V0 *= V0/int(round(E0 - selection)) #TODO Formule à retravailler (vraiment)

    # Pour la luminosité

    E1 += int(round(E1 - selection))/iterations
    V1 *= V1/int(round(E1 - selection)) #TODO Formule à retravailler (vraiment)

    # Pour les mouvements

    idMvt = df(selection, 'movementLabel')
    varFactor = 1.4 # Facteur variation
    for i in range(nombreMouvements):
        if i == idMvt:
            pMvt[i] *= varFactor
        else:
            pMvt[i] *= nombreMouvements/varFactor
    return pMvt, V0, E0, V1, E1

Répartition des tableaux suivant une loi normale (première itération loi uniforme (mais random au début)).

In [13]:
import random as rd

def choix(P, pMvt, V0, E0, V1, E1, iterations):
    k = 5 # 5 tableaux proposés
    trueIndex = []
    draws = [rd.randrange(0, df.shape[0], 1) for i in range(k)]
    IDs = [P[k] for k in draws]

    displayID = 1
    for ID in IDs:
        print("=== Choix #{} ===".format(displayID))
        print(df(ID))
        trueIndex.append(ID)
        displayID+=1
    valid = False # valid input
    while not valid:
        selection = input("Choisissez une oeuvre (0-{})".format(k-1))
        try:
            selection = int(selection)
            if 0 <= selection < k:
                valid = True
        except ValueError:
            continue
    recompute(trueIndex[selection], pMvt, V0, E0, V1, E1, iterations)
    return pMvt, V0, E0, V1, E1

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
import math

def probaDate(E, V, df):
    df.sort_values(by='inception')
    p = 1 - (V/E)
    n = E / p
    P = []
    P[0] = (p**0)*((1-p)**(n-0))*math.factorial(n)/(math.factorial(0) * math.factorial(n - 0))
    for k in range(1, df.shape[0]):
        P[k] = (p**k)*((1-p)**(n-k))*math.factorial(n)/(math.factorial(k) * math.factorial(n - k))
        P[k] += P[k-1]  # Fonction de répartition
    return P

In [ ]:
def probaMouvement(pMvt, countByMvt):
    P = []
    for k in range(1, df.shape[0]):
        idMvt = df(k, 'movementLabel')
        P[k] = pMvt[idMvt]/countByMvt[idMvt]
    return P

In [ ]:
def probaImage(E, V, df): # Vérif que c'est ok
    df.sort_values(by='brightness')
    P = []
    p = 1 - (V/E)
    n = E / p
    P = []
    P[0] = (p**0)*((1-p)**(n-0))*math.factorial(n)/(math.factorial(0) * math.factorial(n - 0))
    for k in range(1, df.shape[0]):
        P[k] = (p**k)*((1-p)**(n-k))*math.factorial(n)/(math.factorial(k) * math.factorial(n - k))
        P[k] += P[k-1]  # Fonction de répartition
    return P

In [ ]:
def verifSeuil():
    # True si le seuil est dépassé
    if V0 < 5 and V1 < 5 and max(pMvt) > 0.4: # TODO Constantes OK???
        return True
    return False

In [ ]:
def probaTotal(pMvt, E0, V0, df):
    P0 = probaDate(E0, V0, df)
    P1 = pMvt
    P2 = probaImage(E0, V0, df)
    PTotal = P0+P1+P2
    PTotal /= 3
    return PTotal

In [ ]:
def trouverArtiste():
    return ""

In [ ]:
nombreMouvements = 10  #TODO faire ça proprement
labelId = {'label1':0, 'label2':1, '3':2, '4':3, '5':4, '6':5, '7':6, '8':7, '9':8, '10':9} # etc.
countByMvt = []
for Id in labelId.values():
    countByMvt[Id] = df.count(level='movementLabel')

In [ ]:
import PIL

def brightness( im_file ):
   im = PIL.Image.open(im_file).convert('L')
   stat = PIL.ImageStat.Stat(im)
   return stat.mean[0]

brightness = []
for painting in df:
    brightness.append(brightness(painting['path'])) # TODO Vérif bon nom
df = df.assign(brightness=brightness)

In [ ]:
# Boucle principale
E0 = 100 # Espérance date
V0 = 20 # Variance date
pMvt = nombreMouvements * [1/nombreMouvements]
iterations = 0
while verifSeuil():
    iterations += 1
    pMvt, V0, E0, V1, E1 = choix(P, pMvt, V0, E0, V1, E1, iterations)
    P = probaTotal(pMvt, E0, V0, E1, V1, df)

trouverArtiste()